In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np

class NN_PreProcessing:

    def __init__(self, X):

        self.dataset = X
        self.norm_dataset = None
        data_min, data_max = None, None

    def prepare_data(self):
        #1. Clean the Data
        self.data_cleaning()

        #2. Normalizing the data
        scaler = MinMaxScaler()
        self.norm_dataset = scaler.fit_transform(self.dataset)

        #3. Deviding input X and output Y
        X = self.norm_dataset[:, :2]
        Y = self.norm_dataset[:, 2:]

        #4. Test and Validation set split
        X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=30)

        #saving min max values to be usined in game to normalize inputs in game
        self.data_min, self.data_max = scaler.data_min_, scaler.data_max_

        self.save_data_to_file(self.data_min, 'Traning_min_values')
        self.save_data_to_file(self.data_max, 'Traning_max_values')

        return X_train, X_val, Y_train, Y_val

    def data_cleaning(self):
        """
        Function to clean the data (custom defined)
        """  
        #all duplicate values for output is removed , even if input is different
        output_slice = self.dataset[:,[2,3]]
        _, unq_row_indices = np.unique(output_slice,return_index=True,axis=0)
        self.dataset = self.dataset[unq_row_indices]
       
        self.dataset = self.dataset[self.dataset[:, 2] != 0]        # removing any rows which has 0 as new x velocity
        self.dataset = self.dataset[self.dataset[:, 3] != 0]        # removing any rows which has 0 as new y velocity

    def save_data_to_file(self, X, filename):

        np.savetxt("{}.csv".format(filename), X, delimiter=",")

In [4]:
class AircraftNN:
    """
    A multilayer Neural Network with one hidden layer
    """

    def __init__(self, X_number, Y_number, hidden_neuron=8, epochs=1000, lambda_val=0.6, learning_rate=0.3,
                 momentum_rate=0.3):
        """
        :param X_number: # of inputs - 2
        :param Y_number: # of outputs - 2
        :param hidden_neuron: # of hidden neuron - 6
        :param epochs: # of epochs - 6
        :param lambda_val: lambda value to be used in sigmoid activation function - 6
        :param learning_rate: learning rate of the model
        :param momentum_rate: momentum rate (for previous gradients)
        """

        # define hyper parameters
        self.vlambda = lambda_val  # 0.6
        self.learningRate = learning_rate  # 0.4
        self.hiddenLayerNeuron = hidden_neuron  # 0.6
        self.mrate = momentum_rate  # 0.2

        # define other given parameters
        self.inputLayerFeatures = X_number
        self.outputLayerNeuron = Y_number
        self.epochs = epochs
        self.train_rmse_error = []
        self.val_rmse_error = []

        # have used seed to keep the random weights initialization contact to test efficiency of hyperparameters
        np.random.seed(5)

        # random initialize the weights
        # size of weights to hidden layer = (hidden neuron * (inputs + 1 (for bias))
        # size of weights to output layer = (hidden neuron * (output + 1 (for bias))
        
#         self.weightHidden = np.random.uniform(-1, 1, size=(self.hiddenLayerNeuron, self.inputLayerFeatures + 1))
#         self.weightOutput = np.random.uniform(-1, 1, size=(self.outputLayerNeuron, self.hiddenLayerNeuron + 1))

        self.weightHidden = np.genfromtxt('weightHidden.csv', delimiter=',')
        self.weightOutput = np.genfromtxt('weightOutput.csv', delimiter=',')
        
        # intializing the gradients to 0 for the first epoch
        self.mGradWeightHidden = np.zeros((self.hiddenLayerNeuron, self.inputLayerFeatures + 1))
        self.mGradWeightOutput = np.zeros((self.outputLayerNeuron, self.hiddenLayerNeuron + 1))

        self.outputHidden = None

    def forward_propagation(self, x):
        """
        Forward Propagation function to calculate predicted output
        """
        # added 1 to input & output of hidden for bias
        input_hidden = np.dot(np.append(x, 1), self.weightHidden.T)
        self.outputHidden = self.sigmoid(input_hidden)
        input_output_layer = np.dot(np.append(self.outputHidden, 1), self.weightOutput.T)
        predicted_output = self.sigmoid(input_output_layer)

        return predicted_output

    def backward_propagation(self, error, x, output_predicted):
        """
            Backward Propagation using gradient descent to update the weights
        """
        gradient_output = self.vlambda * error * self.derivative_sigmoid(output_predicted)
        d_weight_output = (self.learningRate * np.dot(gradient_output.reshape(self.outputLayerNeuron, -1),
                                                      np.append(self.outputHidden, 1).reshape(-1,
                                                                                              self.hiddenLayerNeuron + 1))) + self.mrate * self.mGradWeightOutput
        gradient_hidden = self.vlambda * self.derivative_sigmoid(np.append(self.outputHidden, 1)) * np.dot(
            gradient_output, self.weightOutput)
        d_weight_hidden = self.learningRate * np.dot(gradient_hidden.reshape(self.hiddenLayerNeuron + 1, -1),
                                                     np.append(x, 1).reshape(-1, self.inputLayerFeatures + 1))[:-1,
                                              :] + self.mrate * self.mGradWeightHidden

        self.weightOutput += d_weight_output
        self.weightHidden += d_weight_hidden

        self.mGradWeightOutput = d_weight_output
        self.mGradWeightHidden = d_weight_hidden

    def train_model(self, x_train, y_train, x_val, y_val):
        """
           Trains model running forward & backward propagation
        """
        for ep in range(0, self.epochs):
            train_mse_sum = 0
            val_mse_sum = 0                       

            for i, (input, output) in enumerate(zip(x_train, y_train)):
                expected_output = output
                pred_output = self.forward_propagation(input)
                error = expected_output - pred_output
                self.backward_propagation(error, input, pred_output)

                train_mse = np.mean(error ** 2)
                train_mse_sum += train_mse

            for i, (input, output) in enumerate(zip(x_val, y_val)):
                val_output = self.forward_propagation(input)
                val_error = output - val_output
                val_mse = np.mean(val_error ** 2)
                val_mse_sum += val_mse

            train_mse_error = train_mse_sum / x_train.shape[0]
            self.train_rmse_error.append(np.sqrt(train_mse_error))

            val_mse_error = (val_mse_sum / x_val.shape[0])
            self.val_rmse_error.append(np.sqrt(val_mse_error))
            print("Error at epoch {} is {},Val={}".format(ep+1, np.sqrt(train_mse_error), np.sqrt(val_mse_error)))
            
            if (ep > 1) :
                  if (self.val_rmse_error[ep]-self.val_rmse_error[ep-1])>0.01 :
                        return self.weightHidden, self.weightOutput
         
                
        return self.weightHidden, self.weightOutput
            
            
    def predict_output(self, x_val, y_val):
        """
        Predict Output for given inputs and calculate rmse
        :param x_val:
        :param y_val:
        :return:
        """
        predict_mse_sum = 0
        for i, (input, output) in enumerate(zip(x_val, y_val)):
            predicted_output = self.forward_propagation(input)
            print("input={}\nOutput={}\npredicted_output={}".format(input, output, predicted_output))
            predict_error = output - predicted_output
            predict_mse = np.mean(predict_error ** 2)
            predict_mse_sum += predict_mse

        predict_mse_error_mean = predict_mse_sum / x_val.shape[0]
        return np.sqrt(predict_mse_error_mean)

    def sigmoid(self, x):
        """
            This function returns sigmoid value of input
        """
        return 1 / (1 + np.exp(-x * self.vlambda))

    def derivative_sigmoid(self, x):
        """
            This function returns sigmoid derivative value of input
        """
        return x * (1 - x)
    
    def matrix_dot_product(self,X,Y):
        
        if len(X.shape)<2:
                      
            product=np.zeros((1,Y.shape[1]))
            i=0
            for j in range(Y.shape[1]):
                #print("X[i]={}\nY[:,j].T={}".format(X,Y[:,j].T))
                product[i,j] = np.sum(X*Y[:,j].T)
            #print("product={}".format(product))
            return product
        else:
            product=np.zeros((X.shape[0],Y.shape[1]))
            if(X.shape[1]!=Y.shape[0]):
                raise Exception("Matrix Dimention doesn't Match")
           
            for i in range(X.shape[0]):

                for j in range(Y.shape[1]):
                    product[i,j] = np.sum(X[i]*Y[:,j].T)
            #print("product={}".format(product))
            return product


    def get_weight_bias(self):
        """
            Getter function for weights
        """
        return self.weightHidden, self.weightOutput

    def save_data_to_file(self,file_format):
        """
        Save file in csv format
        """
        if file_format=="csv":            
            np.savetxt("weightHidden.csv", self.weightHidden, delimiter=",")
            np.savetxt("weightOutput.csv", self.weightOutput, delimiter=",")
            
        elif file_format=="npy":
            np.save("weightHidden.npy", self.weightHidden)
            np.save("weightOutput.npy", self.weightOutput)



In [5]:
if __name__ == '__main__':

    # import dataset
    dataset = np.genfromtxt('ce889_dataCollection_new.csv', delimiter=',')

    #creating object for pre processing
    
    pp = NN_PreProcessing(dataset)
    X_train, X_val, Y_train, Y_val = pp.prepare_data()
    
     
    pp.save_data_to_file(np.c_[X_train, Y_train], 'train_data')
    pp.save_data_to_file(np.c_[X_train, Y_train], 'validation_data')
    

    ANN = AircraftNN(X_train.shape[1], Y_train.shape[1])

    # ANN=aircraftNN(inputnumber=items.shape[1],outputnumber=targets.shape[1],HiddenNeuron=5,epoch=100,l=0.7,lr=0.7,mt=0.1)
    ANN.train_model(X_train, Y_train, X_val, Y_val)

    weightHidden, weightOutput = ANN.get_weight_bias()
    print("weightHidden={}\nweightOutput={}".format(weightHidden, weightOutput))
    #ANN.save_data_to_file("csv")




Error at epoch 1 is 0.0911993235314609,Val=0.09132532050611629
Error at epoch 2 is 0.09103294741578881,Val=0.09116346958447
Error at epoch 3 is 0.0908595188433374,Val=0.09099656624193259
Error at epoch 4 is 0.09068169482891966,Val=0.09082686553158387
Error at epoch 5 is 0.09050175089219477,Val=0.09065627494972278
Error at epoch 6 is 0.09032152466737699,Val=0.09048632344789616
Error at epoch 7 is 0.09014244623060805,Val=0.09031817105894026
Error at epoch 8 is 0.08996559255551444,Val=0.09015267320534297
Error at epoch 9 is 0.08979177284202075,Val=0.08999047819938713
Error at epoch 10 is 0.0896216241989763,Val=0.08983212238327146
Error at epoch 11 is 0.08945569235493227,Val=0.08967809779487998
Error at epoch 12 is 0.08929448225876888,Val=0.08952888291287033
Error at epoch 13 is 0.0891384755978808,Val=0.08938493939093911
Error at epoch 14 is 0.08898812115432869,Val=0.08924668531730423
Error at epoch 15 is 0.08884380874081331,Val=0.08911445865100737
Error at epoch 16 is 0.08870583852235213,

Error at epoch 128 is 0.08380123272314363,Val=0.08425354324158979
Error at epoch 129 is 0.08378982975180405,Val=0.08424193409502569
Error at epoch 130 is 0.08377892698472733,Val=0.08423070734033383
Error at epoch 131 is 0.08376847730864913,Val=0.08421981579975664
Error at epoch 132 is 0.0837584374512584,Val=0.08420921788246344
Error at epoch 133 is 0.08374876817501031,Val=0.08419887752431748
Error at epoch 134 is 0.08373943429118741,Val=0.08418876394816045
Error at epoch 135 is 0.08373040453585505,Val=0.08417885129480636
Error at epoch 136 is 0.08372165134690776,Val=0.08416911816995117
Error at epoch 137 is 0.08371315057672964,Val=0.08415954714555172
Error at epoch 138 is 0.08370488116923111,Val=0.08415012424688488
Error at epoch 139 is 0.08369682482396551,Val=0.08414083844926795
Error at epoch 140 is 0.0836889656643488,Val=0.08413168120175668
Error at epoch 141 is 0.08368128992188013,Val=0.0841226459893478
Error at epoch 142 is 0.08367378564407836,Val=0.08411372794039998
Error at epoc

Error at epoch 253 is 0.0832344215760281,Val=0.08357141113921236
Error at epoch 254 is 0.08323191786578335,Val=0.08356866350364697
Error at epoch 255 is 0.08322943032465051,Val=0.08356593439495513
Error at epoch 256 is 0.08322695897688785,Val=0.08356322348517194
Error at epoch 257 is 0.08322450384125252,Val=0.08356053045023383
Error at epoch 258 is 0.08322206493070065,Val=0.08355785497048278
Error at epoch 259 is 0.08321964225215187,Val=0.08355519673116271
Error at epoch 260 is 0.08321723580629198,Val=0.08355255542289895
Error at epoch 261 is 0.0832148455874262,Val=0.0835499307421586
Error at epoch 262 is 0.08321247158338727,Val=0.08354732239168057
Error at epoch 263 is 0.0832101137754758,Val=0.08354473008087891
Error at epoch 264 is 0.08320777213845136,Val=0.08354215352620584
Error at epoch 265 is 0.08320544664055829,Val=0.08353959245147993
Error at epoch 266 is 0.083203137243588,Val=0.08353704658816996
Error at epoch 267 is 0.0832008439029778,Val=0.083534515675636
Error at epoch 268 

Error at epoch 378 is 0.08302218277836389,Val=0.08331930929363467
Error at epoch 379 is 0.08302106619949903,Val=0.0833178405700638
Error at epoch 380 is 0.08301995602097927,Val=0.08331637874043567
Error at epoch 381 is 0.08301885219525018,Val=0.08331492376827387
Error at epoch 382 is 0.08301775467489422,Val=0.08331347561696627
Error at epoch 383 is 0.0830166634126233,Val=0.08331203424976015
Error at epoch 384 is 0.08301557836127704,Val=0.08331059962975912
Error at epoch 385 is 0.08301449947381717,Val=0.08330917171991792
Error at epoch 386 is 0.08301342670332701,Val=0.08330775048304036
Error at epoch 387 is 0.08301236000300541,Val=0.08330633588177612
Error at epoch 388 is 0.0830112993261678,Val=0.08330492787861804
Error at epoch 389 is 0.08301024462624405,Val=0.0833035264359019
Error at epoch 390 is 0.08300919585677975,Val=0.08330213151580335
Error at epoch 391 is 0.08300815297143194,Val=0.0833007430803382
Error at epoch 392 is 0.08300711592397653,Val=0.08329936109136221
Error at epoch 

Error at epoch 503 is 0.08291905165900126,Val=0.08317712097142481
Error at epoch 504 is 0.08291844119894659,Val=0.08317623620290084
Error at epoch 505 is 0.08291783341002873,Val=0.08317535472746454
Error at epoch 506 is 0.08291722828857458,Val=0.08317447654926614
Error at epoch 507 is 0.08291662583114869,Val=0.0831736016730176
Error at epoch 508 is 0.08291602603453761,Val=0.08317273010397329
Error at epoch 509 is 0.08291542889573615,Val=0.08317186184790774
Error at epoch 510 is 0.08291483441193101,Val=0.08317099691109403
Error at epoch 511 is 0.08291424258048473,Val=0.0831701353002823
Error at epoch 512 is 0.0829136533989199,Val=0.08316927702267442
Error at epoch 513 is 0.08291306686490592,Val=0.08316842208590187
Error at epoch 514 is 0.08291248297624094,Val=0.08316757049800118
Error at epoch 515 is 0.08291190173083567,Val=0.0831667222673879
Error at epoch 516 is 0.08291132312669963,Val=0.08316587740283317
Error at epoch 517 is 0.08291074716192237,Val=0.08316503591343638
Error at epoch

Error at epoch 628 is 0.08286186438944604,Val=0.08309363433123396
Error at epoch 629 is 0.08286153656233937,Val=0.08309317415724898
Error at epoch 630 is 0.08286121022684245,Val=0.08309271657142352
Error at epoch 631 is 0.08286088536972538,Val=0.08309226155018279
Error at epoch 632 is 0.08286056197786766,Val=0.08309180906998934
Error at epoch 633 is 0.08286024003825751,Val=0.08309135910735116
Error at epoch 634 is 0.08285991953799128,Val=0.08309091163882852
Error at epoch 635 is 0.08285960046427741,Val=0.08309046664104064
Error at epoch 636 is 0.08285928280443668,Val=0.0830900240906733
Error at epoch 637 is 0.08285896654590044,Val=0.08308958396448501
Error at epoch 638 is 0.08285865167621413,Val=0.0830891462393126
Error at epoch 639 is 0.08285833818303887,Val=0.08308871089207884
Error at epoch 640 is 0.08285802605414713,Val=0.08308827789979684
Error at epoch 641 is 0.08285771527742787,Val=0.08308784723957612
Error at epoch 642 is 0.08285740584088525,Val=0.08308741888862821
Error at epo

Error at epoch 753 is 0.08282927046082057,Val=0.08304992769918704
Error at epoch 754 is 0.08282905894854914,Val=0.08304965159750398
Error at epoch 755 is 0.0828288480052901,Val=0.08304937623540534
Error at epoch 756 is 0.08282863762762825,Val=0.08304910160658278
Error at epoch 757 is 0.08282842781218296,Val=0.08304882770481198
Error at epoch 758 is 0.08282821855560951,Val=0.08304855452395225
Error at epoch 759 is 0.08282800985459395,Val=0.08304828205794447
Error at epoch 760 is 0.08282780170585743,Val=0.0830480103008095
Error at epoch 761 is 0.08282759410615423,Val=0.08304773924664932
Error at epoch 762 is 0.08282738705227007,Val=0.08304746888964322
Error at epoch 763 is 0.08282718054102162,Val=0.08304719922404791
Error at epoch 764 is 0.08282697456925962,Val=0.0830469302441969
Error at epoch 765 is 0.08282676913386511,Val=0.08304666194449772
Error at epoch 766 is 0.08282656423174889,Val=0.08304639431943286
Error at epoch 767 is 0.0828263598598538,Val=0.08304612736355735
Error at epoch

Error at epoch 878 is 0.08280640978677269,Val=0.08301973530129135
Error at epoch 879 is 0.08280625071234446,Val=0.08301952073226201
Error at epoch 880 is 0.08280609194999101,Val=0.08301930649926081
Error at epoch 881 is 0.0828059334984925,Val=0.08301909260074095
Error at epoch 882 is 0.08280577535663652,Val=0.08301887903516707
Error at epoch 883 is 0.08280561752321708,Val=0.08301866580101662
Error at epoch 884 is 0.08280545999703712,Val=0.0830184528967794
Error at epoch 885 is 0.08280530277690501,Val=0.08301824032095781
Error at epoch 886 is 0.08280514586163802,Val=0.08301802807206574
Error at epoch 887 is 0.0828049892500607,Val=0.08301781614862873
Error at epoch 888 is 0.08280483294100269,Val=0.08301760454918426
Error at epoch 889 is 0.08280467693330233,Val=0.08301739327228153
Error at epoch 890 is 0.0828045212258044,Val=0.0830171823164804
Error at epoch 891 is 0.0828043658173602,Val=0.0830169716803525
Error at epoch 892 is 0.0828042107068281,Val=0.0830167613624801
Error at epoch 893 

In [ ]:
#ANN.save_data_to_file("csv")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1000),ANN.train_rmse_error)
plt.plot(range(1000),ANN.val_rmse_error)
plt.title('Errors')
plt.ylabel('rmse')
plt.xlabel('epoch')


In [ ]:
plt.plot(range(999),x)
plt.title('Errors')
plt.ylabel('rmse')
plt.xlabel('epoch')

In [ ]:
x=[]
for i in range(1,1000):
    x.append(ANN.val_rmse_error[i] - ANN.train_rmse_error[i])
min(x)
x.index(0.00036496906570063214)

In [ ]:
def matrix_dot_product(X,Y):
    print("X.shape={}\nY.shape={}".format(X.shape,Y.shape))
    if(X.shape[1]!=Y.shape[0]):
        raise Exception("Matrix Dimention doesnt Match")
    product=np.zeros((X.shape[0],Y.shape[1]))
    for i in range(X.shape[0]):
        print("X[i]={}".format(X[i]))    
        for j in range(Y.shape[1]):
            print(j)
            print("Y[:,j]={}".format(Y[:,j])) 
            product[i,j] = np.sum(X[i]*Y[:,j].T)
    return product

 